# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686561


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:  10%|█         | 3/30 [00:07<01:11,  2.64s/it]

Rendering models:  13%|█▎        | 4/30 [00:08<00:53,  2.07s/it]

Rendering models:  17%|█▋        | 5/30 [00:09<00:45,  1.82s/it]

Rendering models:  20%|██        | 6/30 [00:10<00:34,  1.43s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:21<00:01,  2.47it/s]

Rendering models:  93%|█████████▎| 28/30 [00:21<00:01,  1.76it/s]

Rendering models:  97%|█████████▋| 29/30 [00:22<00:00,  2.14it/s]

Rendering models: 100%|██████████| 30/30 [00:22<00:00,  2.26it/s]

zac8151                               0.000360
jammiedodgers                         0.000659
smithkurtis                           0.000273
ctiborekskutr                         0.000290
CThomas                               0.002135
not-logged-in-82c06f1bd03315127bbc    0.004010
KG.1612                               0.003580
kcarte13                              0.000366
jmfranci                              0.000947
tingard                               0.000337
Kimby12                               0.000230
not-logged-in-468edef11712cfc1e24c    0.002371
Stella07                              0.000213
AlexYoung35                           0.000248
pangeli5                              0.000636
not-logged-in-d9c9c60cbb19848c1e87    0.000543
acapirala                             0.002034
rhondacard                            0.001233
Lavadude                              0.014789
not-logged-in-7397e17c7492ef1cdfe6    0.000244
not-logged-in-c0c1a76a597edbb84ded    0.000286
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())